In [3]:
import pandas_profiling
import pandas as pd

In [7]:
# We start by importing the new data set
home_folder = '/Users/anthonymiyoro/Documents/code/DataTho/'

#Read dataset
train_modified = pd.read_csv(home_folder + 'train_modified.csv')
test_bundas = pd.read_csv(home_folder + 'bundas_test.csv')

### Use pandas profiling to get summary details of the one-hot-encoded dataset

In [8]:
profile = pandas_profiling.ProfileReport(train_modified)
profile.to_file(output_file= home_folder + "/html/train_modified.htm")

## Splitting into train and test

To understand how well our model generalizes, we need to split our data into a test and training set. In order to get a true prediction, we can't show our model the test labels. Instead we just ask the model to score based upon the test explanatory variables.
In order to get to this, we split the dataframe into a test and train.


### X_train, y_train, X_test, y_test

However, we additionally need to seperate the explanatory features from our outcome feature.
To do this we need to create four subsets of data:
X_train: the explanatory features to train the algorithm
y_train: the outcome feature associated with the training features - In this case, these are the loan amounts
X_test: the explanatory features to test the algorithm
y_test: the true Y of the target features associated with the testing geatures - again, in this case, these are the loan amounts.

To do this we will use the sklearn model selection function train_test_split. If you look back to the top of this page, we imported this function at the beginning of the notebook.
The names used below (X_train, X_test, y_train, y_test) are the conventions used in industry for train and test data so you will continue to see them repeatedly.